# Import Required Libraries
Import the necessary libraries, including pathlib, datetime, and the custom modules.

In [1]:
# Import the necessary libraries
import pandas as pd
from pathlib import Path
import re
import sys
sys.path.append('src')

from datetime import datetime, timedelta

# Import custom modules
from data.loader import BankDataLoader
import config
from analysis.reports import *

# Load and Filter Data
Load the bank statements data using BankDataLoader and filter out transactions before a specified date.

In [2]:
# Load the bank statements data using BankDataLoader
loader = BankDataLoader(config.DATA_DIRS)
data = loader.load_statements()

# Filter out transactions before a specified date
filter_date = '2023-02-01'
data = data[data['Date'] >= filter_date]

# Dump the intermediate data to a CSV file
loader.dump_intermediate(data, 'intermediate.csv')

# Display the first few rows of the filtered data
data.head()

,Date,Description,Amount,Income,Expense,Category,Category1,Category2,Category3,Category12,Category123,Account,Tag
2544,2023-02-01,Gallipott,-78.90,0.0,-78.90,Expense:Miscellaneous:hair cuts,Expense,Miscellaneous,hair cuts,Expense:Miscellaneous,Expense:Miscellaneous:hair cuts,Discover,NaN
1073,2023-02-01,American Family Insurance,-257.05,0.0,-257.05,Expense:Insurance,Expense,Insurance,Unassigned,Expense:Insurance,Expense:Insurance:Unassigned,First Tech,NaN
2545,2023-02-01,HILTON ADVPURCH800236711 MEMPHIS TN,-136.16,0.0,-136.16,Expense:Unassigned:Unassigned,Expense,Unassigned,Unassigned,Expense:Unassigned,Expense:Unassigned:Unassigned,Discover,None
2546,2023-02-02,Amazon,-6.83,0.0,-6.83,Expense:Miscellaneous,Expense,Miscellaneous,Unassigned,Expense:Miscellaneous,Expense:Miscellaneous:Unassigned,Discover,NaN
2547,2023-02-02,SHOWTIX4U* FCHS TOWER 7029898987 NV,-56.95,0.0,-56.95,Expense:Unassigned:Unassigned,Expense,Unassigned,Unassigned,Expense:Unassigned,Expense:Unassigned:Unassigned,Discover,None


# Filter Data

In [3]:
data = data[~data['Tag'].fillna('').str.contains('#exclude_investment', flags=re.IGNORECASE, regex=True)]
data = data[~data['Tag'].fillna('').str.contains('#ccpayment', flags=re.IGNORECASE, regex=True)]
data = data[~data['Tag'].fillna('').str.contains('#colege529', flags=re.IGNORECASE, regex=True)]
data = data[~data['Tag'].fillna('').str.contains('#IBondRedemption', flags=re.IGNORECASE, regex=True)]
data = data[~data['Tag'].fillna('').str.contains('#exclude_car', flags=re.IGNORECASE, regex=True)]
data = data[~data['Tag'].fillna('').str.contains('#exclude_Janet529', flags=re.IGNORECASE, regex=True)]


# Dump Intermediate Data
Dump the filtered data to an intermediate CSV file.

In [4]:
# Dump the intermediate data to a CSV file
loader.dump_intermediate(data, 'intermediate.csv')

# Display the first few rows of the filtered data
# data.head()

# Generate Monthly Totals Report
Generate and display the monthly totals report using the monthly_totals function.

In [5]:
# Generate Monthly Totals Report
monthly_totals(data)

Total for 2023-02:  -1,486.56 (Income:  10,072.87, Expense: -11,559.43)
Total for 2023-03:   3,656.99 (Income:  15,155.19, Expense: -11,498.20)
Total for 2023-04:  -2,330.90 (Income:   8,811.47, Expense: -11,142.37)
Total for 2023-05:  -3,124.25 (Income:  11,020.18, Expense: -14,144.43)
Total for 2023-06:    -107.66 (Income:   8,758.51, Expense:  -8,866.17)
Total for 2023-07:    -389.69 (Income:  11,823.20, Expense: -12,212.89)
Total for 2023-08:  -2,007.63 (Income:   9,127.89, Expense: -11,135.52)
Total for 2023-09:  -1,709.58 (Income:  12,123.74, Expense: -13,833.32)
Total for 2023-10:  -1,173.63 (Income:  11,552.61, Expense: -12,726.24)
Total for 2023-11:     943.98 (Income:  11,562.29, Expense: -10,618.31)
Total for 2023-12:   9,463.89 (Income:  20,624.44, Expense: -11,160.55)
Total for 2024-01:  -3,650.14 (Income:  12,820.87, Expense: -16,471.01)
Total for 2024-02:      -5.69 (Income:   9,464.60, Expense:  -9,470.29)
Total for 2024-03:  -1,737.65 (Income:   9,465.00, Expense: -11,

In [ ]:
# Pivot?
df = data

df['Date'] = pd.to_datetime(df['Date'])
df['Month'] = df['Date'].dt.to_period('M')
df['Amount'] = df['Amount'].fillna(0)

# Group by month
# df['Amount']

df_group = df.groupby([pd.Grouper(key='Month'), 'Category12'])['Amount'].sum().reset_index()
# Ensure every month/category12 pair has a valid value, which is 0 if NaN
df_group = df_group.pivot(index='Month', columns='Category12', values='Amount').fillna(0).reset_index()
df_group = df_group.melt(id_vars=['Month'], var_name='Category12', value_name='Amount')

# with pd.option_context('display.max_rows', None):
#     print(df_group) 
# df_group

# # Calculate the rolling average
df_group['Rolling_Avg'] = df_group.groupby('Category12')['Amount'].transform(lambda x: x.rolling(window=12, min_periods=1).mean())

# # Pivot to create a column for each category
df_group = df_group.pivot(index='Category12', columns='Month', values='Rolling_Avg').reset_index()
df_group = df_group.round(2)

# Filter columns to show only months after 2024-01
df_group = df_group[['Category12'] + [col for col in df_group.columns if not str(col).startswith('Category') and str(col) >= '2024-01']]
df_group


AttributeError: 'Period' object has no attribute 'startswith'

In [9]:
df = rolling_12M_avg_category(data, 'Category1')
df

Month,Category1,Category1,2024-01,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,2024-11,2024-12
0,Expense,Expense,-12095.8,-11929.15,-11908.16,-11839.97,-11725.41,-12038.07,-12152.65,-12352.13,-12471.45,-12473.59,-12621.35,-12833.74
1,Income,Income,11936.2,11892.96,11422.41,11701.43,11702.04,11853.08,11665.63,11677.96,11724.03,11540.06,11503.64,11217.65
